In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
version = 'v2'

# R  W  1111111111111
# R  P  111111111111
# G  W  1111111111111
# G  P  111111111111
# H  W  111111111111
# H  P  111111111111

# category = 'R'
category = 'G'
# category = 'H'

bet_type = 'W'
# bet_type = 'P'

In [3]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

38252 races loaded!


In [4]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

15664 G races!


In [5]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
16,47,SANDOWN PARK,VIC,M,G,2017-10-01,3,EDITHVALE BOWLING CLUB HT3,2017-10-01 03:30:00,Paying,515,"[[2], [3], [4], [6]]",8.0,"[{""runnerName"": ""BARWELL BOY"", ""runnerNumber"":...",14.4,18.6,186.6,477.6,"[[2], [3], [4], [6]]","[{'runnerName': 'BARWELL BOY', 'runnerNumber':..."
17,49,SANDOWN PARK,VIC,M,G,2017-10-01,5,THIRD MILITARY GOLF GROUP,2017-10-01 04:11:00,Paying,515,"[[3], [6], [4], [2]]",6.0,"[{""runnerName"": ""LIKELY TARGET"", ""runnerNumber...",36.7,209.7,1972.0,1694.7,"[[3], [6], [4], [2]]","[{'runnerName': 'LIKELY TARGET', 'runnerNumber..."
18,52,SANDOWN PARK,VIC,M,G,2017-10-01,8,KEYSBOROUGH PARK CC,2017-10-01 05:07:00,Paying,595,"[[6], [2, 7], [], [5]]",8.0,"[{""runnerName"": ""GABONIA"", ""runnerNumber"": 1, ...",16.7,38.7,59.4,867.9,"[[6], [2, 7], [], [5]]","[{'runnerName': 'GABONIA', 'runnerNumber': 1, ..."


In [6]:
# extract runners from races
data_all = []
for ri, race in dfo.iterrows():
    data_all.extend(race['runners'])
data_all = pd.DataFrame(data_all)
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,fpo,fpp,...,runnerNumber,tpo,tpp,trainerName,two,twp,win_odds,win_perc,win_rank,win_scaled
143277,NaN,NaN,NaN,NaN,4,0,0,"{'returnWin': 6, 'returnWinOpen': 1.01, 'retur...",NaN,NaN,...,4,NaN,NaN,D WALSH,NaN,NaN,3.2,0.312500,1.000000,0.264321
143278,NaN,NaN,NaN,NaN,5,0,0,"{'returnWin': 7.5, 'returnWinOpen': 1.01, 'ret...",NaN,NaN,...,5,NaN,NaN,S A CAHILL,NaN,NaN,11.9,0.084034,0.166667,0.071078
143279,NaN,NaN,NaN,NaN,6,0,0,"{'returnWin': 4.6, 'returnWinOpen': 1.01, 'ret...",NaN,NaN,...,6,NaN,NaN,W R WRIGHTING,NaN,NaN,4.3,0.232558,0.666667,0.196704


In [7]:
# drop scratched
data = data_all.dropna(subset=['has_odds'])
data.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,fpo,fpp,...,runnerNumber,tpo,tpp,trainerName,two,twp,win_odds,win_perc,win_rank,win_scaled
143277,NaN,NaN,NaN,NaN,4,0,0,"{'returnWin': 6, 'returnWinOpen': 1.01, 'retur...",NaN,NaN,...,4,NaN,NaN,D WALSH,NaN,NaN,3.2,0.312500,1.000000,0.264321
143278,NaN,NaN,NaN,NaN,5,0,0,"{'returnWin': 7.5, 'returnWinOpen': 1.01, 'ret...",NaN,NaN,...,5,NaN,NaN,S A CAHILL,NaN,NaN,11.9,0.084034,0.166667,0.071078
143279,NaN,NaN,NaN,NaN,6,0,0,"{'returnWin': 4.6, 'returnWinOpen': 1.01, 'ret...",NaN,NaN,...,6,NaN,NaN,W R WRIGHTING,NaN,NaN,4.3,0.232558,0.666667,0.196704


In [8]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    143280.000000
mean          0.109618
std           0.312414
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: finishingPosition, dtype: float64

In [9]:
xn = data['num_runners']
xwp = data['win_perc']
xws = data['win_scaled']
xwr = data['win_rank']
xpp = data['place_perc']
xps = data['place_scaled']
xpr = data['place_rank']
X = pd.concat([xn, xwp, xws, xwr, xpp, xps, xpr], axis=1)
X.describe()

,num_runners,win_perc,win_scaled,win_rank,place_perc,place_scaled,place_rank
count,143280.000000,143280.000000,143280.000000,143280.000000,143280.000000,143280.000000,143280.000000
mean,0.131779,0.129594,0.109324,0.528908,0.360736,0.109324,0.534142
std,0.014373,0.136499,0.114896,0.327453,0.257208,0.079438,0.330170
min,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.125000,0.031746,0.026697,0.300000,0.175439,0.052869,0.300000
50%,0.125000,0.086957,0.073352,0.555556,0.344828,0.103375,0.600000
75%,0.125000,0.188679,0.159408,0.800000,0.526316,0.161239,0.800000
max,0.333333,0.961538,0.893574,1.000000,1.000000,0.762795,1.000000


In [10]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 500
print('epochs = {}'.format(epochs))

layer_1 = 50
layer_2 = 50

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 7
epochs = 500
tag = G50x50W
model loaded


In [11]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)
print('model saved')


model saved
